In [12]:
import pandas as pd

from emissions_parameters import DIR_EMISSIONS_RATES, \
    DIR_DR_POTENTIAL_HRS, EMISSIONS_SCENARIO_LIST, EMISSIONS_RATES_FILES, \
    DR_HRS_FILES, DR_POTENTIAL_FILES, DR_SEASONS

def create_emissions_factors_df():
    """
    Given a list of emissions rate files for different policy scenarios, 
    creates a dataframe with 2022-2041 hourly emissions factors for all policy scenarios.
    """
    sheet = 'HourlyAvoidedEmissionsRate'
    columns = ['Report_Year','Report_Month','Report_Day','Report_Hour', \
               'Emissions Rate Estimate']   
    
    for idx,file_name in enumerate(EMISSIONS_RATES_FILES):
        
        xlsx = pd.ExcelFile(DIR_EMISSIONS_RATES+file_name)
        column_name = EMISSIONS_SCENARIO_LIST[idx] + ' Emissions Rate Estimate'
        
        #for first file, read in hours 
        if idx == 0: 
            df = pd.read_excel(xlsx,sheet,usecols=columns[0:3])  
        else:
            pass 

        #for all files, add emissions rates to existing dataframe
        df[column_name] = pd.read_excel(xlsx,sheet,usecols=[columns[4]])
    
    #subset year 2022-2041 #when testing make sure this matches year range for other vars
    df_subset = df[df['Report_Year']>=2022]
    
    return df_subset

dfshow = create_emissions_factors_df()

,Report_Year,Report_Month,Report_Day,Baseline Emissions Rate Estimate,EarlyCoalRetirement Emissions Rate Estimate,LimitedMarkets Emissions Rate Estimate,NoGasBuildLimits Emissions Rate Estimate,OrgMarkets Emissions Rate Estimate,SCC Emissions Rate Estimate
0,2021,1,1,0.780794,0.908502,0.912884,0.764419,0.891376,0.390961
1,2021,1,1,1.020618,1.027524,1.073576,1.100273,1.018774,0.451447
2,2021,1,1,1.015126,1.084103,1.140878,1.142728,1.082675,0.453014
3,2021,1,1,1.041399,1.069614,1.253425,1.162150,1.199418,0.452223
4,2021,1,1,0.981788,0.940068,1.215418,1.145416,1.304543,0.477398
...,...,...,...,...,...,...,...,...,...
184075,2041,12,31,1.039660,NaN,NaN,NaN,NaN,NaN
184076,2041,12,31,0.990830,NaN,NaN,NaN,NaN,NaN
184077,2041,12,31,0.957687,NaN,NaN,NaN,NaN,NaN
184078,2041,12,31,0.835334,NaN,NaN,NaN,NaN,NaN


In [58]:
import pandas as pd

from emissions_calculator.emissions_parameters import DIR_EMISSIONS_RATES, \
    DIR_DR_POTENTIAL_HRS, EMISSIONS_SCENARIO_LIST, EMISSIONS_RATES_FILES, \
    DR_NAME, DR_HRS_FILES, DR_POTENTIAL_FILES, SUBSET_PRODUCTS, DR_SEASONS

def create_DR_hours_df():
    """
    Input: Excel files with DR hours, where each file contains separate sheets for each season, 
    and each sheet contains columns for the DR products in that season. 
    Output: A dictionary of dataframes where each dataframe gives the DR hours for a given DR plan 
    and season within that plan. 
    """
    DR_hours_df_dict = {} 

    #when testing, should have same number of dr hrs files and dr potential files
    for idx, file_name in enumerate(DR_HRS_FILES):
        
        print(idx)
        drname = DR_NAME[idx]
        print(DR_SEASONS[idx])
        seasons = DR_SEASONS[idx]
        xlsx = pd.ExcelFile(DIR_DR_POTENTIAL_HRS+file_name)
        
        for season in seasons:
            dict_key = drname + '_' + season
            DR_hours_df_dict[dict_key] = pd.read_excel(xlsx,season)
        
    return DR_hours_df_dict

dr_hrs_dict = create_DR_hours_df()
for key in dr_hrs_dict.keys():
    print(key)
    df = dr_hrs_dict[key]
df

0
['Winter', 'Summer']
1
['Winter', 'Summer', 'Fall']
oldbins_Winter
oldbins_Summer
newbins_Winter
newbins_Summer
newbins_Fall


,hourID,Month,Day,DVR,ResTOU
0,1,1,1,0,0
1,2,1,1,0,0
2,3,1,1,0,0
3,4,1,1,0,0
4,5,1,1,0,0
...,...,...,...,...,...
8755,20,12,31,0,0
8756,21,12,31,0,0
8757,22,12,31,0,0
8758,23,12,31,0,0


In [49]:
xlsx = pd.ExcelFile(DIR_DR_POTENTIAL_HRS+DR_POTENTIAL_FILES[0])
#files oriented with summer potential first, then winter potential
summer_df = pd.read_excel(xlsx,'Reporter Outputs',index_col=0,header=None,skiprows=1,nrows=21,usecols=range(21)).T
winter_df = pd.read_excel(xlsx,'Reporter Outputs',index_col=0,header=None,skiprows=26,nrows=18,usecols=range(21)).T

#for new binning, only use resTOU and DVR

In [2]:
import pandas as pd
import numpy as np

from emissions_parameters import DIR_EMISSIONS_RATES, \
    DIR_DR_POTENTIAL_HRS, EMISSIONS_SCENARIO_LIST, EMISSIONS_RATES_FILES, \
    DR_NAME, DR_HRS_FILES, DR_POTENTIAL_FILES, SUBSET_PRODUCTS, DR_SEASONS

def create_DR_potential_df_dict():
    """
    Input: Excel files containing DR potential for each year 2022-2041 within each season.
    Output: A dictionary of dataframes where each dataframe gives the potential for a given DR plan 
    and season within that plan.
    """
    DR_potential_df_dict = {}
    
    for idx, file_name in enumerate(DR_POTENTIAL_FILES):
        
        drname = DR_NAME[idx]
        xlsx = pd.ExcelFile(DIR_DR_POTENTIAL_HRS+file_name)
       
        # in these files, summer and winter are in the same sheet, summer first and then winter
        # come back to make less hard-coded
        dict_key = drname + '_Summer'
        DR_potential_df_dict[dict_key] = pd.read_excel(xlsx,'Reporter Outputs',index_col=0,header=None,skiprows=1,nrows=21,usecols=range(21)).T
        DR_potential_df_dict[dict_key] = DR_potential_df_dict[dict_key].rename(columns={'Product': 'Year'})
        
        dict_key = drname + '_Winter'
        DR_potential_df_dict[dict_key] = pd.read_excel(xlsx,'Reporter Outputs',index_col=0,header=None,skiprows=26,nrows=19,usecols=range(21)).T
        DR_potential_df_dict[dict_key] = DR_potential_df_dict[dict_key].rename(columns={'Product': 'Year'})

        #if only a subset of products is desired
        subset = SUBSET_PRODUCTS[idx].copy()
        if isinstance(subset[0],str):
            subset.insert(0,'Year')
            print(subset)
            DR_potential_df_dict[drname + '_Summer'] = DR_potential_df_dict[drname + '_Summer'][subset]
            DR_potential_df_dict[drname + '_Winter'] = DR_potential_df_dict[drname + '_Winter'][subset]
        else:
            pass
        
        seasons = DR_SEASONS[idx]
        if 'Fall' in seasons: #for new bins, winter potential is applied to fall
            DR_potential_df_dict[drname + '_Fall'] = DR_potential_df_dict[drname + '_Winter']
        else:
            pass
        
    return DR_potential_df_dict

dr_pot_dict = create_DR_potential_df_dict()
for key in dr_pot_dict.keys():
    print(key)
    df_pot = dr_pot_dict[key]
    print(df_pot)

['Year', 'DVR', 'ResTOU']
oldbins_Summer
0     Year  NRCurtailCom  NRCurtailInd     NRIrrLg  NRIrrSmMed         DVR  \
1   2022.0      6.939121     32.751443   55.898246   65.880076   50.464615   
2   2023.0     13.781884     65.925864  113.229468  133.449016  100.305452   
3   2024.0     20.575588     99.810273  177.928958  209.701986  151.066354   
4   2025.0     27.426883    134.458233  242.202518  285.452967  203.203128   
5   2026.0     34.341287    169.192912  306.470422  361.197283  250.847001   
6   2027.0     34.753889    169.669692  309.795257  365.115839  311.186474   
7   2028.0     34.283612    169.836450  314.361494  370.497475  361.737147   
8   2029.0     34.327169    169.938125  318.322978  375.166366  407.512096   
9   2030.0     34.673740    170.224107  324.347639  382.266860  456.210488   
10  2031.0     34.970273    170.853831  329.157189  387.935259  516.772159   
11  2032.0     35.308090    171.233126  333.512006  393.067722  530.747618   
12  2033.0     35.21435

In [4]:
def create_product_info_df():
    """
    Input: Excel files containing DR potential for each year 2022-2041 within each season.
    Output: A dictionary of dataframes for each DR plan, listing products, bins, seasonality, shift or shed 
    """
    DR_product_info_df_dict = {}
    
    for idx, file_name in enumerate(DR_POTENTIAL_FILES):
        
        drname = DR_NAME[idx]
        xlsx = pd.ExcelFile(DIR_DR_POTENTIAL_HRS+file_name)
        column_names = ['Product','Bin','Seasonality','Shift or Shed?']
        DR_product_info_df_dict[drname] = pd.read_excel(xlsx,'EnergyCalcs', \
                                                       skiprows=2,nrows=23,usecols=column_names)  
        
    return DR_product_info_df_dict

dr_pot_dict = create_product_info_df()
for key in dr_pot_dict.keys():
    print(key)
    df_pot = dr_pot_dict[key]
    print(df_pot)

oldbins
           Product    Bin Seasonality Shift or Shed?
0     NRCurtailCom  Bin 2  Year-round          Shift
1     NRCurtailInd  Bin 2  Year-round          Shift
2          NRIrrLg  Bin 1      Summer           Shed
3       NRIrrSmMed  Bin 1      Summer           Shed
4           ComCPP  Bin 1  Year-round           Shed
5           IndCPP  Bin 1  Year-round           Shed
6              DVR  Bin 1  Year-round           Shed
7           IndRTP  Bin 1  Year-round          Shift
8           ResCPP  Bin 1  Year-round           Shed
9           ResTOU  Bin 2  Year-round          Shift
10   NRCoolSwchMed  Bin 2      Summer          Shift
11   NRHeatSwchMed  Bin 2      Winter          Shift
12    NRCoolSwchSm  Bin 4      Summer          Shift
13    NRHeatSwchSm  Bin 3      Winter          Shift
14       NRTstatSm  Bin 3  Year-round          Shift
15       ResACSwch  Bin 4      Summer          Shift
16   ResHeatSwitch  Bin 3      Winter          Shift
17         ResBYOT  Bin 2  Year-round 

In [42]:
for a in range(20):
    print(a)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [4]:
subset = ['DVR','ResTOU']
subset.insert(0,'Year')

In [10]:
subset = SUBSET_PRODUCTS[1].copy()
subset
 #       if isinstance(subset[0],str):
 #           subset.insert(0,'Year')
 #           print(subset)

['Year', 'Year', 'Year', 'DVR', 'ResTOU']